In [1]:
using Dates
using ArgParse
using Printf
using Random
using GaussianProcesses

include("../btg.jl")

credible_interval (generic function with 3 methods)

In [ ]:
# load creep data
df = DataFrame(CSV.File("../datasets/creeprupt/taka", header=0))
data = convert(Matrix, df[:,[1;3:end]])
target = convert(Array, df[:, 2]);

In [14]:
# shuffle data
ind_shuffle = randperm(MersenneTwister(1234), size(data, 1)) 
data = data[ind_shuffle, :]
target = target[ind_shuffle]
# training set
id_train = 1:400; posx = 1:30; posc = 1:30; n_train = length(id_train)
x = data[id_train, posx] 
Fx = data[id_train, posc] 
y = float(target[id_train])
ymax_train = maximum(y)
y ./= ymax_train
trainingData0 = trainingData(x, Fx, y) #training data used for testing various functions
d = getDimension(trainingData0); n = getNumPts(trainingData0); p = getCovDimension(trainingData0)

30

In [15]:
#parameter setting
rangeθ = [0.125 1000]
rangeθm = repeat(rangeθ, d, 1)
rangeλ = [-1.5 1.] #we will always used 1 range scale for lambda
myquadtype = ["QuasiMonteCarlo", "QuasiMonteCarlo"]

2-element Array{String,1}:
 "QuasiMonteCarlo"
 "QuasiMonteCarlo"

In [16]:
# build btg model
btg0 = btg(trainingData0, rangeθm, rangeλ; quadtype = myquadtype)
(pdf0_raw, cdf0_raw, dpdf0_raw, quantInfo0_raw) = solve(btg0); 

In [17]:
id_test = 801:810
n_test = length(id_test)
id_fail = []
id_nonproper = []
x_test = data[id_test, posx]
Fx_test = data[id_test, posc]
y_test_true = target[id_test]
count_test = 0
error_abs = 0.
error_sq = 0.
nlpd = 0.

for i in 1:n_test
    global error_abs, error_sq, nlpd, count_test
    before = Dates.now()
    x_test_i = reshape(x_test[i, :], 1, length(posx))
    Fx_test_i = reshape(Fx_test[i, :], 1, length(posc))
    try
        pdf_test_i, cdf_test_i, dpdf_test_i, quantbound_test_i, support_test_i = pre_process(x_test_i, Fx_test_i, pdf0_raw, cdf0_raw, dpdf0_raw, quantInfo0_raw)
        y_test_i_true = y_test_true[i]
        median_test_i = ymax_train * quantile(cdf_test_i, quantbound_test_i, support_test_i)[1]
        try 
            CI_test_i = ymax_train .* credible_interval(cdf_test_i, quantbound_test_i, support_test_i; mode=:equal, wp=.95)[1]
            count_test += (y_test_i_true >= CI_test_i[1])&&(y_test_i_true <= CI_test_i[2]) ? 1 : 0
            # @info "CI" CI_test_i
        catch err
            append!(id_fail, i)
        end
        error_abs += abs(y_test_i_true - median_test_i)
        error_sq += (y_test_i_true - median_test_i)^2
        nlpd += log(pdf_test_i(y_test_i_true)) 
#         @info y_test_i_true, median_test_i
#         @info error_abs, error_sq
#         @info count_test, id_fail, id_nonproper
    catch err 
        append!(id_nonproper, i)
    end
    after = Dates.now()
    elapsedmin = round(((after - before) / Millisecond(1000))/60, digits=5)
    @info elapsedmin
    end

┌ Info: 0.45437
└ @ Main In[17]:40
┌ Info: 0.43962
└ @ Main In[17]:40
┌ Info: 0.44352
└ @ Main In[17]:40
┌ Info: 0.45322
└ @ Main In[17]:40
┌ Info: 0.469
└ @ Main In[17]:40
┌ Info: 0.45097
└ @ Main In[17]:40
┌ Info: 0.45892
└ @ Main In[17]:40
┌ Info: 0.43608
└ @ Main In[17]:40
┌ Info: 0.47312
└ @ Main In[17]:40
┌ Info: 0.44462
└ @ Main In[17]:40


In [18]:
count_test /= n_test - length(id_fail)
error_abs  /= n_test
error_sq   /= n_test
nlpd       /= -n_test
@info error_abs, error_sq, nlpd, count_test

┌ Info: (28.933933258870564, 1717.331746149653, 124.41866306740683, 0.9)
└ @ Main In[18]:5


In [19]:
# training set
x = data[id_train, posx]' 
y = float(target[id_train])
# build and fit a GP
mymean = MeanZero(); kern = SE(zeros(d),0.0) 
gp = GP(x, y, mymean, kern) 
optimize!(gp)     
# predict
x_test = data[id_test, posx]'
y_test_true = target[id_test]
μ, σ² = predict_y(gp, x_test); stdv = sqrt.(σ²)
error_GP = abs.(μ .- y_test_true)
error_abs_GP = mean(error_GP)
error_sq_GP = mean(error_GP.^2)
CI_test_GP = hcat(-1.96.*stdv .+ μ, 1.96.*stdv .+ μ)
count_test_GP = sum((y_test_true .>= CI_test_GP[:, 1]) .* (y_test_true .<= CI_test_GP[:,2]))/n_test
nlpd_GP = -mean(log.(pdf.(Normal(), (y_test_true.-μ)./stdv)./stdv))

12.789828967769768

In [ ]:
error_GP

In [20]:
@info error_abs_GP, error_sq_GP, nlpd_GP, count_test_GP

┌ Info: (56148.149261290164, 5.24943465870176e9, 12.789828967769768, 1.0)
└ @ Main In[20]:1


In [24]:
x = data[id_train, posx]' 
y = float(target[id_train])
trans = BoxCox()
g_fixed(x) = trans(x, 0.); dg(x) = partialx(trans, x, 0.)
invg(x) = inverse(trans, x, 0.)
gy = g_fixed.(y) 
# build and fit a GP
mymean = MeanZero(); kern = SE(zeros(d),0.0) 
loggp = GP(x, gy, mymean, kern) 
optimize!(loggp) 
# predict
x_test = data[id_test, posx]'
y_test_true = target[id_test]
μ, σ² = predict_y(loggp, x_test); stdv = sqrt.(σ²)
CI_test_logGP = invg.(hcat(-1.96.*stdv .+ μ, 1.96.*stdv .+ μ))
count_test_logGP = sum((y_test_true .>= CI_test_logGP[:, 1]) .* (y_test_true .<= CI_test_logGP[:,2]))/n_test
y_pred = invg.(μ)
error_logGP = abs.(y_pred .- y_test_true)
error_abs_logGP = mean(error_logGP)
error_sq_logGP = mean(error_logGP.^2)
nlpd_logGP = -mean(log.( dg.(y_test_true) .* pdf.(Normal(), (g_fixed.(y_test_true).-μ)./stdv) ./stdv ))

InterruptException: InterruptException:

In [25]:
@info error_abs_logGP, error_sq_logGP, nlpd_logGP, count_test_logGP

┌ Info: (47.98384467132985, 3829.6654631976817, 5.349362513912765, 1.0)
└ @ Main In[25]:1
